# Precalculated data for WDPA: biodiversity - only birds
## Setup
### Library import
We import all the required Python libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy
from itertools import repeat
import functools

### Utils

**getHTfromId**

In [2]:
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

**format_df**

In [3]:
def format_df(path, file_name, lookups_id):
    #save_steps= np.arange(1,217486,step=5000)
    ### Loop over each set of data and add all results to one dictionary that will be a text field in the Feature Service

    df = pd.read_csv(f'{path}/{file_name}')
    col_name = [col for col in df.columns if col in ['amphibians','birds','presence','reptiles']]
    df.rename(columns={'SliceNumbe':'SliceNumber',col_name[0]:'SUM'}, inplace=True)

    ### Get species area against global species range:
    lookup = getHTfromId(lookups_id)
    df = df.merge(lookup[['SliceNumber','range_area_km2']], how='left',on = 'SliceNumber')
    df['per_global'] = round(df['SUM']/df['range_area_km2']*100,2)
    df.loc[df['per_global']> 100,'per_global'] = 100 ### make max presence 100%
    
    ### Get species area against aoi area:
    df = df.merge(wdpa[['MOL_ID','AREA_KM2']])
    df['per_aoi'] = round(df['SUM']/df['AREA_KM2']*100,2)
    df.loc[df['per_aoi']> 100,'per_aoi'] = 100 ### make max presence 100%
    
    return df

## Connect to ArcGIS API

In [4]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [5]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [6]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


## Read data

In [2]:
path_in = '/Users/sofia/Documents/HE_Data/WDPA_Precalculated/Inputs'
path_out = '/Users/sofia/Documents/HE_Data/WDPA_Precalculated/Outputs'

### WDPA

In [8]:
# wdpa table
wdpa= gpd.read_file(f'zip://{path_in}/WDPA_FILTERED_20210615_FILTERED_nomarine_with_oecm_wdpa.zip')
wdpa.rename(columns={'AREA_KM':'AREA_KM2'},inplace=True)

In [38]:
dff = wdpa[['NAME','MOL_ID','AREA_KM2']].copy() ## remove geometry here because it is the non-simplified table
dff.head()

,NAME,MOL_ID,AREA_KM2
0,Boulder Beach,1,1.136031
1,Ferndale,2,0.748492
2,Broughton Bay,3,0.031907
3,Kaipupu Point,4,0.270855
4,Catlins Conservation Park,5,8.412168


### WDPA's biodiversity data

In [28]:
### Ids of lookup tables for each taxa in ArcGIS online

lookups = {'amphibians':'c221a727e12048b2a6ec8e762bc5f478',
         'birds':'bcb31fd9091446a0af3cfdaed334a8da',
         'mammals':'212a3dd4665845deb5d2adf5b597aae0',
         'reptiles':'5b606a03b3fc431e8d4b9191c88bc2b9'}

# Get data for all taxa
amphibians = format_df(path_in, 'wdpa_amphibians_final_20211003.csv', lookups['amphibians'])
birds = format_df(path_in, 'wdpa_birds_final_20211003.csv', lookups['birds'])
mammals = format_df(path_in, 'wdpa_mammals_final_20211003.csv', lookups['mammals'])
reptiles = format_df(path_in, 'wdpa_reptiles_final_20211003.csv', lookups['reptiles'])

In [29]:
amphibians.head(1)

,OBJECTID,MOL_ID,X,Y,SUM,SliceNumber,Dimensions,range_area_km2,per_global,AREA_KM2,per_aoi
0,1,102,173.412225,-41.161038,1.0,3318,SliceNumber,296432,0.0,0.008886,100.0


In [31]:
birds.head(1)

,OID_,MOL_ID,X,Y,SUM,SliceNumber,Dimensions,range_area_km2,per_global,AREA_KM2,per_aoi
0,1,121,169.34608,-44.652219,1.0,251.0,SliceNumber,12099814,0.0,1.93288,51.74


In [32]:
mammals.head(1)

,OID_,MOL_ID,X,Y,SUM,SliceNumber,Dimensions,range_area_km2,per_global,AREA_KM2,per_aoi
0,1,131,172.506219,-41.971634,2.0,813.0,SliceNumber,86298,0.0,0.718592,100.0


In [33]:
reptiles.head(1)

,OBJECTID,MOL_ID,X,Y,SUM,SliceNumber,Dimensions,range_area_km2,per_global,AREA_KM2,per_aoi
0,1,121,169.34608,-44.652219,1.0,6163,SliceNumber,137822,0.0,1.93288,51.74


## Processing

In [34]:
amphibians = amphibians.groupby('MOL_ID')[['SliceNumber', 'per_global', 'per_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('amphibians').reset_index()
birds = birds.groupby('MOL_ID')[['SliceNumber', 'per_global', 'per_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('birds').reset_index()
mammals = mammals.groupby('MOL_ID')[['SliceNumber', 'per_global', 'per_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('mammals').reset_index()
reptiles = reptiles.groupby('MOL_ID')[['SliceNumber', 'per_global', 'per_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('reptiles').reset_index()


In [36]:
amphibians.head(1)

,MOL_ID,amphibians
0,2,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi..."


In [39]:
dff = pd.merge(dff, amphibians, how='left', on = 'MOL_ID')
dff = pd.merge(dff, birds, how='left', on = 'MOL_ID')
dff = pd.merge(dff, mammals, how='left', on = 'MOL_ID')
dff = pd.merge(dff, reptiles, how='left', on = 'MOL_ID')
dff.head()

,NAME,MOL_ID,AREA_KM2,amphibians,birds,mammals,reptiles
0,Boulder Beach,1,1.136031,NaN,"[{""SliceNumber"":482.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",NaN
1,Ferndale,2,0.748492,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi..."
2,Broughton Bay,3,0.031907,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":1847.0,""per_global"":0.0,""per_a...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6499,""per_global"":0.0,""per_aoi..."
3,Kaipupu Point,4,0.270855,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi..."
4,Catlins Conservation Park,5,8.412168,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.01,""per_aoi...",NaN,"[{""SliceNumber"":6163,""per_global"":0.01,""per_ao..."


In [15]:
dff.loc[dff['MOL_ID']==121,'birds'].values[0]

'[{"SliceNumber":251.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":552.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":613.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1301.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1310.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1321.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1510.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1511.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1517.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1572.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1933.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":2017.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":3025.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":3027.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":3081.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":3277.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":3422.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":3552.0,"per_global":0.0,"per_aoi":

**Save table**

In [41]:
dff.to_csv((f'{path_out}/wdpa_precalculated_biodiversity_only.csv'))

### Get contextual data from old table

In [42]:
ctx = pd.read_csv(f'{path_in}/all_precalc_wdpa_no_geom.csv')
ctx.head(1)

,Unnamed: 0,NAME,MOL_ID,AREA_KM,reptiles,amphibians,mammals,birds,population_sum,majority_land_cover_climate_regime,...,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3
0,0,Boulder Beach,1,1.136031,[],[],"[{""SliceNumber"": 303.0, ""percentage_presence"":...","[{""SliceNumber"": 482.0, ""percentage_presence"":...",2.110001,107.0,...,NaN,NaN,100.0,NaN,310492.0,Stewardship Area,Stewardship Area,National,III,NZL


In [47]:
ctx.columns

Index(['Unnamed: 0', 'NAME', 'MOL_ID', 'AREA_KM', 'reptiles', 'amphibians',
       'mammals', 'birds', 'population_sum',
       'majority_land_cover_climate_regime', 'land_cover_majority',
       'climate_regime_majority', 'percent_irrigated', 'percent_rainfed',
       'percent_rangeland', 'percent_urban', 'WDPAID', 'DESIG', 'DESIG_E',
       'DESIG_T', 'IUCN_CA', 'ISO3'],
      dtype='object')

In [59]:
ctx2 = ctx[['MOL_ID','population_sum','majority_land_cover_climate_regime', 'land_cover_majority','climate_regime_majority',
            'percent_irrigated', 'percent_rainfed','percent_rangeland', 'percent_urban',]].copy() 

In [60]:
ctx2.head()

,MOL_ID,population_sum,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban
0,1,2.110001,107.0,Grassland,Cool Temperate Moist,NaN,NaN,100.00,NaN
1,2,1.315837,176.0,Forest,Warm Temperate Moist,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,100.00,NaN
4,5,3.103363,97.0,Forest,Cool Temperate Moist,NaN,NaN,71.43,NaN


### Merge contextual data to WDPA dataframe

In [61]:
dff2 = pd.merge(dff, ctx2, how='left', on = 'MOL_ID')
dff2.head()

,NAME,MOL_ID,AREA_KM2,amphibians,birds,mammals,reptiles,population_sum,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban
0,Boulder Beach,1,1.136031,NaN,"[{""SliceNumber"":482.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",NaN,2.110001,107.0,Grassland,Cool Temperate Moist,NaN,NaN,100.00,NaN
1,Ferndale,2,0.748492,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",1.315837,176.0,Forest,Warm Temperate Moist,NaN,NaN,NaN,NaN
2,Broughton Bay,3,0.031907,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":1847.0,""per_global"":0.0,""per_a...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6499,""per_global"":0.0,""per_aoi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kaipupu Point,4,0.270855,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",NaN,NaN,NaN,NaN,NaN,NaN,100.00,NaN
4,Catlins Conservation Park,5,8.412168,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.01,""per_aoi...",NaN,"[{""SliceNumber"":6163,""per_global"":0.01,""per_ao...",3.103363,97.0,Forest,Cool Temperate Moist,NaN,NaN,71.43,NaN


In [62]:
dff2.to_csv((f'{path_out}/wdpa_precalculated_with_contextual.csv'))